In [1]:
import apk_image as converter
from sklearn.preprocessing import StandardScaler
import numpy as np

In [68]:
data, labels = converter.convert()
labels=np.where(labels==0.0, 0, 1)
n=1550
total_apps = labels.shape[0]
scaler_model = StandardScaler()

In [69]:
data=np.where(data<100, data, 0)
data_aux=np.array(data, copy=True)
data=data[:n,:]
labels_aux=np.array(labels, copy=True)
labels=labels[:n,:]

In [70]:
fold_size = int(n / 5)

feature_columns = [str(i) for i in range(153)]
feature_cols = [tf.feature_column.numeric_column(k) for k in feature_columns]

acc_sum = 0.0
model = tf.estimator.DNNClassifier([15], feature_columns=feature_cols, optimizer='Adagrad')
print('number of apps taken {}'.format([n]))
for i in range(5):
    train_set = {
        'data': np.concatenate((data[0:i * fold_size, :], data[i * fold_size + fold_size:, :]), axis=0),
        'labels': np.concatenate((labels[0:i * fold_size, :], labels[i * fold_size + fold_size:, :]), axis=0)
    }
    test_set = {
        'data': data[i * fold_size:i * fold_size + fold_size, :],
        'labels': labels[i * fold_size:i * fold_size + fold_size:, :]
    }

    train_images = np.array(train_set['data'], copy=True)
    test_images = np.array(test_set['data'], copy=True)

    scaler_model.fit(train_images)
    scaler_model.transform(train_images)
    scaler_model.transform(test_images)

    feature_dict_train = {str(i): train_images[:, i:i + 1] for i in range(153)}
    feature_dict_test = {str(i): test_images[:, i:i + 1] for i in range(153)}
    train_input_func = tf.estimator.inputs.numpy_input_fn(x=feature_dict_train, y=train_set['labels'],
                                                          num_epochs=None, shuffle=True,
                                                          batch_size=train_images.shape[0])
    eval_input_func = tf.estimator.inputs.numpy_input_fn(x=feature_dict_test, y=test_set['labels'], num_epochs=1,
                                                         shuffle=False, batch_size=test_images.shape[0])
    print('fold number {}'.format(i))
    acc = 0
    prev_acc = -1
    while acc > prev_acc:
        model.train(train_input_func, steps=100)
        results = model.evaluate(eval_input_func)
        prev_acc = acc
        acc = results['accuracy']
    acc_sum += prev_acc
    print('accuracy is {}'.format(prev_acc))

average_accuracy = acc_sum / 5

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_summary_steps': 100, '_num_ps_replicas': 0, '_service': None, '_model_dir': '/tmp/tmp761x1xl8', '_is_chief': True, '_keep_checkpoint_every_n_hours': 10000, '_task_id': 0, '_tf_random_seed': None, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_num_worker_replicas': 1, '_session_config': None, '_log_step_count_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe7e8cbfc18>, '_master': ''}
number of apps taken [1550]
fold number 0
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmp761x1xl8/model.ckpt.
INFO:tensorflow:step = 1, loss = 956.642
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmp761x1xl8/model.ckpt.
INFO:tensorflow:Loss for final step: 32.8823.
INFO:tensorflow:Starting evaluation at 2018-01-14-15:37:16
INFO:tensorflow:Restoring parameters 

accuracy is 1.0
fold number 4
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmp761x1xl8/model.ckpt-800
INFO:tensorflow:Saving checkpoints for 801 into /tmp/tmp761x1xl8/model.ckpt.
INFO:tensorflow:step = 801, loss = 8.84794
INFO:tensorflow:Saving checkpoints for 900 into /tmp/tmp761x1xl8/model.ckpt.
INFO:tensorflow:Loss for final step: 7.93523.
INFO:tensorflow:Starting evaluation at 2018-01-14-15:44:34
INFO:tensorflow:Restoring parameters from /tmp/tmp761x1xl8/model.ckpt-900
INFO:tensorflow:Finished evaluation at 2018-01-14-15:44:35
INFO:tensorflow:Saving dict for global step 900: accuracy = 1.0, accuracy_baseline = 0.870968, auc = 1.0, auc_precision_recall = 1.0, average_loss = 0.0102278, global_step = 900, label/mean = 0.870968, loss = 3.17063, prediction/mean = 0.867375
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmp761x1xl8/model.ckpt-900
INFO:tensorflow:Saving checkpoints for 901 into /tmp/t

In [71]:
average_accuracy

0.99225807189941406

In [72]:
train_images=np.array(data_aux[1551:1700,:], copy=True)

scaler_model.fit(train_images)
scaler_model.transform(train_images)
predict_input_func = tf.estimator.inputs.numpy_input_fn(x={str(i): train_images[:, i:i + 1] for i in range(153)}, y=labels_aux[1551:1700,:],
                                                          num_epochs=1, shuffle=False,
                                                          batch_size=train_images.shape[0])
p=model.predict(predict_input_func)

In [99]:
p=list(p)

In [100]:
a=np.random.permutation(range(len(p)))

In [101]:
acc=[p[i]['class_ids'][0] for i in range(len(p))]

In [102]:
labels_aux[1551:1700].shape

(149, 1)

In [103]:
len(acc)

149

In [104]:
acc=np.array(acc)

In [106]:
labels_=np.reshape(labels_aux[1551:1700], -1)

In [112]:
acc==labels_

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

In [108]:
acc

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])

In [109]:
labels_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0])